In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", None)

In [3]:
df = pd.read_csv("diet_daily_raw_export.csv")
df.head()

,data,carboidratos_g,proteinas_g,gorduras_g,kcal,cardio_semanal_min,cicle,peso_kg,observacoes
0,2023-05-06,250,190,85,2525,210,1Cutting I (01/05/2023 - 17/09/2023),104.3,NaN
1,2023-05-07,250,190,85,2525,210,1Cutting I (01/05/2023 - 17/09/2023),104.3,NaN
2,2023-05-08,250,190,85,2525,210,1Cutting I (01/05/2023 - 17/09/2023),104.3,NaN
3,2023-05-09,250,190,85,2525,210,1Cutting I (01/05/2023 - 17/09/2023),104.3,NaN
4,2023-05-10,250,190,85,2525,210,1Cutting I (01/05/2023 - 17/09/2023),104.3,NaN


In [4]:
df.shape

(982, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   data                982 non-null    object 
 1   carboidratos_g      982 non-null    int64  
 2   proteinas_g         982 non-null    int64  
 3   gorduras_g          982 non-null    int64  
 4   kcal                982 non-null    int64  
 5   cardio_semanal_min  982 non-null    int64  
 6   cicle               982 non-null    object 
 7   peso_kg             982 non-null    float64
 8   observacoes         173 non-null    object 
dtypes: float64(1), int64(5), object(3)
memory usage: 69.2+ KB


In [6]:
# Colunas
df.columns

Index(['data', 'carboidratos_g', 'proteinas_g', 'gorduras_g', 'kcal',
       'cardio_semanal_min', 'cicle', 'peso_kg', 'observacoes'],
      dtype='object')

In [7]:
# Granularidade
df.duplicated(subset=["data"]).sum()

np.int64(0)

In [8]:
# Conversão de data
df["data"] = pd.to_datetime(df["data"], errors="coerce")

In [9]:
df[df["data"].isna()]

,data,carboidratos_g,proteinas_g,gorduras_g,kcal,cardio_semanal_min,cicle,peso_kg,observacoes


Nenhuma data invalida

In [10]:
# Valores ausentes
df.isna().sum()

data                    0
carboidratos_g          0
proteinas_g             0
gorduras_g              0
kcal                    0
cardio_semanal_min      0
cicle                   0
peso_kg                 0
observacoes           809
dtype: int64

##### Validações de Dominio

In [11]:
# Macros
numeric_cols = [
    "carboidratos_g",
    "proteinas_g",
    "gorduras_g",
    "kcal",
    "cardio_semanal_min",
    "peso_kg"
]

(df[numeric_cols] < 0).sum()

carboidratos_g        0
proteinas_g           0
gorduras_g            0
kcal                  0
cardio_semanal_min    0
peso_kg               0
dtype: int64

In [12]:
df["kcal_estimado"] = (
    df["carboidratos_g"] * 4 +
    df["proteinas_g"] * 4 +
    df["gorduras_g"] * 9
)

df[["kcal", "kcal_estimado"]].head(10)

,kcal,kcal_estimado
0,2525,2525
1,2525,2525
2,2525,2525
3,2525,2525
4,2525,2525
5,2525,2525
6,2525,2525
7,2425,2425
8,2425,2425
9,2425,2425


In [13]:
(df["kcal"] - df["kcal_estimado"]).abs().describe()

count    982.000000
mean       0.020367
std        0.638226
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       20.000000
dtype: float64

In [14]:
df["cicle"].unique()

array(['1Cutting I (01/05/2023 - 17/09/2023)',
       '2Reversa I (18/09/2023 - 29/10/2023)',
       '3Cutting II (30/10/2023 - 23/02/2024)',
       '4Reversa II (24/02/2024 - 22/03/2024)',
       '5Bulking I (23/03/2024 - 19/05/2024)',
       '6Cutting III (20/05/2024 - 28/09/2024)',
       '7Bulking II (28/10/2024 - 01/01/2025)',
       '8Cutting IV (02/01/2025 - 17/04/2025)',
       '9Manutenção I (18/04/2025 - 05/10/2025)',
       '10Cutting V (06/10/2025 - XX/XX/XXXX)'], dtype=object)

In [15]:
cycle_map = {
    "1Cutting I (01/05/2023 - 17/09/2023)": 1,
    "2Reversa I (18/09/2023 - 29/10/2023)": 2,
    "3Cutting II (30/10/2023 - 23/02/2024)": 3,
    "4Reversa II (24/02/2024 - 22/03/2024)": 4,
    "5Bulking I (23/03/2024 - 19/05/2024)": 5,
    "6Cutting III (20/05/2024 - 28/09/2024)": 6,
    "7Bulking II (28/10/2024 - 01/01/2025)": 7,
    "8Cutting IV (02/01/2025 - 17/04/2025)": 8,
    "9Manutenção I (18/04/2025 - 05/10/2025)": 9,
    "10Cutting V (06/10/2025 - XX/XX/XXXX)": 10
}

df["cycle_id"] = df["cicle"].map(cycle_map)

In [16]:
df[df["cycle_id"].isna()][["data", "cicle"]]

,data,cicle


In [17]:
df = df.rename(columns={
    "data": "diet_date",
    "carboidratos_g": "carbs_g",
    "proteinas_g": "protein_g",
    "gorduras_g": "fat_g",
    "kcal": "calories_kcal",
    "cardio_seamanal_min": "cardio_weekly_min",
    "peso_kg": "bodyweight_kg",
    "observacoes": "notes"
})

In [18]:
df = df.drop(columns=["cicle", "kcal_estimado"])


In [19]:
df = df.replace({np.nan: None})


In [20]:
df.to_csv(
    "diet_daily_clean.csv",
    index=False,
    encoding="utf-8"
)


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   diet_date           982 non-null    datetime64[ns]
 1   carbs_g             982 non-null    int64         
 2   protein_g           982 non-null    int64         
 3   fat_g               982 non-null    int64         
 4   calories_kcal       982 non-null    int64         
 5   cardio_semanal_min  982 non-null    int64         
 6   bodyweight_kg       982 non-null    float64       
 7   notes               173 non-null    object        
 8   cycle_id            982 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(6), object(1)
memory usage: 69.2+ KB


In [22]:
df.tail()

,diet_date,carbs_g,protein_g,fat_g,calories_kcal,cardio_semanal_min,bodyweight_kg,notes,cycle_id
977,2026-01-07,242,190,37,2061,200,82.5,None,10
978,2026-01-08,236,190,38,2046,200,82.5,None,10
979,2026-01-09,175,190,60,2000,200,82.1,None,10
980,2026-01-10,175,190,60,2000,200,82.1,None,10
981,2026-01-11,175,190,60,2000,200,82.1,None,10


In [23]:
df.tail()

,diet_date,carbs_g,protein_g,fat_g,calories_kcal,cardio_semanal_min,bodyweight_kg,notes,cycle_id
977,2026-01-07,242,190,37,2061,200,82.5,None,10
978,2026-01-08,236,190,38,2046,200,82.5,None,10
979,2026-01-09,175,190,60,2000,200,82.1,None,10
980,2026-01-10,175,190,60,2000,200,82.1,None,10
981,2026-01-11,175,190,60,2000,200,82.1,None,10
